In [37]:
import pandas as pd
import requests
import io
import threading
from PIL import Image
import numpy as np
from random import randint
from multiprocessing import Pool
from tqdm import tqdm

In [ ]:
with open("./datasets/conceptual-12m/cc12m.tsv", "r", encoding="utf-8") as dataset_file:
    urls, texts = [], []

    for i, item in enumerate(tqdm(dataset_file, total=12423374)):
        url, text = item.split("\t")

        urls.append(url)
        texts.append(text)

In [ ]:
urls = pd.DataFrame({
    "url": urls,
    "caption": texts
})

In [32]:
urls["filename"] = ["-1"] * len(urls)

In [38]:
def download_image_buffer(i, url):
    global urls
    try:
        data = requests.get(url, headers={"user-agent": "I am a valid user, please give me image"}).content
        img = Image.open(io.BytesIO(data))
        filename = f"{i}.jpg"
        img.save(f"./datasets/conceptual-12m/images/{filename}")
    except:
        filename = "unavailable"
    
    urls.loc[i, "filename"] = filename

In [ ]:
_max = 10000
i = 0

for i, url in enumerate(urls["url"].values[:_max]):
    if urls["filename"][i] == "-1":
        filename = download_image_buffer(i, url)
        urls.loc[i, "filename"] = filename